In [1]:
# delete this cell
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [2]:
!pip install torch transformers accelerate -q

In [3]:
!pip install -q --upgrade gradio

In [13]:
!pip install --user "huggingface_hub[cli]"
!pip install --user "huggingface-hub"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [15]:
!pip install --upgrade huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [7]:
!huggingface-cli login

^C


In [3]:
!huggingface-cli whoami

UjjawalRoy


In [1]:
import torch
import transformers

c:\Users\admin\Downloads\GenAI-POC\Generative-AI-main\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#https://huggingface.co/NousResearch/Llama-2-7b-chat-hf/tree/main
model= "NousResearch/Llama-2-7b-chat-hf"
tokenizer= transformers.AutoTokenizer.from_pretrained(model)

In [11]:
llama_pipeline= transformers.pipeline(
    "text-generation",
    model= model,
    torch_dtype= torch.float16,
    device_map= "auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.85it/s]
c:\Users\admin\Downloads\GenAI-POC\Generative-AI-main\.conda\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\admin\Downloads\GenAI-POC\Generative-AI-main\.conda\Lib\site-packages\transformers\generation\configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file

In [12]:
structure_of_llama_2_prompt= '''
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
'''

In [13]:
system_prompt= """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>

"""

In [14]:
# Llama don't remember the previous prompts unlike chatgpt
# this is why it is necessary to send the history of prompts alomg with the current prompt
def format_message(message: str, history: list, memory_limit: int= 3)-> str:
  if len(history) > memory_limit:
    history= history[-memory_limit:]
  if len(history)== 0:
    return system_prompt+ "{} [/INST]".format(message)

  formatted_message= system_prompt+ "{0} [/INST] {1} </s>".format(history[0][0], history[0][1])
  for user_message, model_answer in history[1:]:
    formatted_message+= "<s>[INST] {0} [/INST] {1} </s>".format(user_message, model_answer)
  formatted_message+= "<s>[INST] {} [/INST]".format(message)

  return formatted_message


In [15]:
def get_responses(message: str, history: list)-> str:
  query= format_message(message, history)
  response= ""
  sequences= llama_pipeline(
      query,
      do_sample= True,
      num_return_sequences= 1,
      eos_token_id= tokenizer.eos_token_id,
      max_length= 1024
  )

  generated_text= sequences[0]['generated_text']
  # response includes both query and model's output
  # remove query
  response= generated_text[len(query):]
  print("Chatbot:", response.strip())
  return response.strip()


In [16]:
import gradio as gr
gr.ChatInterface(get_responses).launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
